In [ ]:
# for abstract modules exposure
import sys

sys.path.append("../../src/")

for line in sys.path:
     print(line)

CZML DATA PROCESSER FOR NAV DATA

In [ ]:
from czml.nav import NavCZMLDataProcess

Testings:

In [ ]:
obj = NavCZMLDataProcess()

In [ ]:
# ingest
data = obj.ingest("./temp/goesrplt_naver2_IWG1_20170322-0136.txt")
data

In [ ]:
pre_processed_data = obj.preprocess(data)
pre_processed_data

In [ ]:
czml_str = obj.prep_visualization(pre_processed_data)
czml_str

Save the string to a new file

In [ ]:
import os

tempdir = 'temp' 
if not os.path.exists(tempdir):
    os.makedirs(tempdir)
    
with open("temp/nav_czml.czml", "w") as czml_file:
    czml_file.write(czml_str)

In [ ]:
# remove at the end of the notebook; needed for module exposure
sys.path.remove("../../src/")


Display in Cesium


In [ ]:
# for abstract modules exposure
import sys

sys.path.append("../../")

for line in sys.path:
     print(line)

In [ ]:
from cesiumviz.czmlviz import CZMLViz

In [ ]:
czml_viz_obj = CZMLViz()
nav_czml_cesium_html = czml_viz_obj.generate_html("./temp/nav_czml.czml")

In [ ]:
print(nav_czml_cesium_html)

In [ ]:
import IPython.display

IPython.display.HTML(data = nav_czml_cesium_html)

In [ ]:
# remove at the end of the notebook; needed for module exposure
sys.path.remove("../../")